In [23]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import pandas as pd
import torch
import transformers
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI

load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
eng_responses = []
eng_data = pd.read_excel("eng_data.xlsx")
#eng_data_sampled = eng_data.groupby("level", group_keys=False).apply(lambda x: x.sample(min(len(x), 1000 // eng_data["level"].nunique())))

model = "gpt-4o-mini"
client = OpenAI(api_key=openai_api_key)
system_prompt = "You are a helpful assistant aiding in the generation of a Q&A dataset. When given a text, your task is to generate a prompt that the text could have been a response to."
chat_history = [{"role": "system", "content": system_prompt}]

for _, row in eng_data.iterrows():
    chat_history = [{"role": "system", "content": system_prompt}]
    text = row["text"]
    prompt = f"""Given the following text, please generate a prompt that it could be a response for:
                \"{text}\"
                """
    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,
        messages=chat_history
    )

    reply = response.choices[0].message.content.strip()
    eng_responses.append(reply)
eng_data["prompt"] = eng_responses
eng_data.to_excel("eng_data_pseudoprompt.xlsx", index=False)

C:\Users\lenao\AppData\Local\Temp\ipykernel_14480\3608878914.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eng_data_sampled = eng_data.groupby("level", group_keys=False).apply(lambda x: x.sample(min(len(x), 1000 // eng_data["level"].nunique())))


ValueError: Length of values (10760) does not match length of index (994)

In [36]:
(rows, cols) = eng_data.shape

In [37]:
eng_data["prompt"] = eng_responses[len(eng_responses) - rows:]

In [38]:
eng_data.to_excel("eng_data_pseudoprompt.xlsx", index=False)